In [62]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Option
import re
import pandas as pd

In [49]:
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)

In [50]:
url = 'https://www.kaggle.com/code/m1nty03/rice-type-classification' 
name = url.split("www.kaggle.com/code/")
name = name[-1].replace("/", "--")
driver.get(url)

In [70]:
iframe = driver.find_element(By.ID, 'rendered-kernel-content')
iframe_src = iframe.get_attribute('src')
response = requests.get(iframe_src)
soup = BeautifulSoup(response.text, 'html.parser')

In [84]:
pattern1 = r'Epoch\s*(\d+)/\d+.*? - accuracy:\s*(\d+\.\d+)\s*- loss:\s*(\d+\.\d+)'
pattern2 = r'Epoch\s*(\d+)/\d+.*? - accuracy:\s*(\d+\.\d+)\s*- loss:\s*(\d+\.\d+)(?:.*? - val_accuracy:\s*(\d+\.\d+)\s*- val_loss:\s*(\d+\.\d+))?'
matches = re.findall(pattern2, soup.text, re.DOTALL)

In [90]:
structured_data = []
for match in matches:
    epoch = int(match[0])
    acc = float(match[1])
    loss = float(match[2])
    val_acc = float(match[3]) if match[3] else None
    val_loss = float(match[4]) if match[4] else None
    structured_data.append((epoch, acc, loss, val_acc, val_loss))

# Create a DataFrame for better readability
df = pd.DataFrame(structured_data, columns=['Epoch', 'Accuracy', 'Loss', 'Val_Accuracy', 'Val_Loss'])
df.to_csv(name + ".csv", index=False)


In [86]:
driver.quit()